In [1]:
from __future__ import annotations

import numpy as np
import pandas as pd

from collections import Counter
from typing import (
    List,
    Dict,
    Optional,
)
import re

import matplotlib.pyplot as plt
%matplotlib inline


Constants

In [2]:
RANDOM_SEED = 5
DATA_PATH = "raw_data"
AK_FILE = "AnnaKarenina"
WNP_RU_FILE = "WarAndPeace"
WNP_EN_FILE = "WarAndPeaceEng"
TXT = ".txt"

np.random.seed(RANDOM_SEED)


Data loading

In [3]:
wnp_ru = None
with open(f"{DATA_PATH}/{WNP_RU_FILE}{TXT}", "r", encoding="utf-8") as f:
    wnp_ru = f.read()
assert wnp_ru is not None


wnp_eng = None
with open(f"{DATA_PATH}/{WNP_EN_FILE}{TXT}", "r", encoding="utf-8") as f:
    wnp_eng = f.read()
assert wnp_eng is not None


ak = None
with open(f"{DATA_PATH}/{AK_FILE}{TXT}", "r", encoding="utf-8") as f:
    ak = f.read()
assert ak is not None


# 1. Реализуйте базовый частотный метод по Шерлоку Холмсу:
* подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
* возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
* расшифруйте их таким частотным методом.


In [4]:
class TextPreproccesing:
    def __init__(self, text: str, ru_en: str = "ru") -> None:
        try:
            self.alph = "А-Яа-я" if ru_en == "ru" else "A-Za-z"
            self.alphabet = (
                "абвгдеёжзийклмнопрстуфхцчшщъыьэюя"
                if ru_en == "ru"
                else "abcdefghijklmnopqrstuvwxyz"
            )
            self.regex = re.compile(f"[\W_\d]+|[^{self.alph}]+")
            self.original_text = self.regex.sub(" ", text.lower()).strip()
        except BaseException as e:
            print(f"error with text processing")
            print(e)

    def count_letters_freq(self):
        self.counter: Optional[Counter] = Counter(self.original_text)
        for letter in self.alphabet:
            if letter not in self.counter.keys():
                self.counter[letter] = 0
        self.letter_freq_seq = [
            letter
            for letter, freq in sorted(
                self.counter.items(),
                key=lambda x: x[1],
                reverse=True,
            )
        ]
        self.alphabet_len = len(self.alphabet)

        return self

    def decode(self, text_preprocessed: TextPreproccesing) -> str:
        assert self.alph == text_preprocessed.alph
        decode_dict = {
            self.letter_freq_seq[i]: text_preprocessed.letter_freq_seq[i]
            for i in range(self.alphabet_len)
        }
        return "".join(map(lambda x: decode_dict[x], text_preprocessed.original_text))


In [5]:
%%time


wnp_ru_processed = TextPreproccesing(wnp_ru).count_letters_freq()
print(wnp_ru_processed.alphabet_len)
print(wnp_ru_processed.original_text[:100*4])
print(wnp_ru_processed.letter_freq_seq)


33
война и мир самый известный роман льва николаевича толстого как никакое другое произведение писателя отражает глубину его мироощущения и философии эта книга из разряда вечных потому что она обо всем о жизни и смерти о любви и чести о мужестве и героизме о славе и подвиге о войне и мире первый том знакомит с высшим обществом россии  века показаны взаимоотношения между родителями и детьми в семье ро
[' ', 'о', 'а', 'е', 'и', 'н', 'т', 'с', 'л', 'в', 'р', 'к', 'д', 'м', 'у', 'п', 'я', 'г', 'ь', 'ы', 'з', 'б', 'ч', 'й', 'ж', 'ш', 'х', 'ю', 'ц', 'э', 'щ', 'ф', 'ъ', 'ё']
CPU times: user 129 ms, sys: 8.01 ms, total: 137 ms
Wall time: 137 ms


In [6]:
class TextEncode:
    """class for ceaser encoding"""

    def __init__(self, shift: int = 1, ru_en: str = "ru") -> None:
        self.alph = "А-Яа-я" if ru_en == "ru" else "A-Za-z"
        self.alphabet = (
            "абвгдеёжзийклмнопрстуфхцчшщъыьэюя"
            if ru_en == "ru"
            else "abcdefghijklmnopqrstuvwxyz"
        )
        self.shift = shift % len(self.alphabet)

        self.encode_dict = {
            origin_letter: self.alphabet[(i + self.shift) % 33]
            for i, origin_letter in enumerate(self.alphabet)
        }
        self.encode_dict[" "] = " "
        self.decode_dict = {v: k for k, v in self.encode_dict.items()}

    def encode(self, text: str) -> str:
        return "".join(map(lambda x: self.encode_dict[x], text))

    def decode(self, text: str) -> str:
        return "".join(map(lambda x: self.decode_dict[x], text))


In [7]:
class TestTexts:
    Kolmogorov_wiki = """Андрей Николаевич Колмогоров (12 (25) апреля 1903, Тамбов — 20 октября 1987, Москва) — советский математик, один из крупнейших математиков XX века. Один из основоположников современной теории вероятностей, им получены фундаментальные результаты в топологии, геометрии, математической логике, классической механике, теории турбулентности, теории сложности алгоритмов, теории информации, теории функций, теории тригонометрических рядов, теории меры, теории приближения функций, теории множеств, теории дифференциальных уравнений, теории динамических систем, функциональном анализе и в ряде других областей математики и её приложений. Автор новаторских работ по философии, истории, методологии и преподаванию математики, известны его работы в статистической физике (в частности, уравнение Джонсона — Мела — Аврами — Колмогорова).

    Профессор Московского государственного университета (с 1931), доктор физико-математических наук, академик Академии наук СССР (1939). Президент Московского математического общества (ММО) в 1964—1966 и 1974—1985 годах. Герой Социалистического Труда (1963). Лауреат Ленинской и Сталинской премии.

    Иностранный член Национальной академии наук США (1967)[8], Лондонского королевского общества (1964)[9], Французской (Парижской) академии наук (1966)[10], член Германской академии естествоиспытателей «Леопольдина» (1959), почётный член Американской академии искусств и наук (1959), иностранный член Венгерской академии наук (1965), Польской академии наук (1956), Нидерландской королевской академии наук (1963), АН ГДР (1977), Академии наук Финляндии (1985), почётный член Румынской академии. Член Лондонского математического общества (1962), Индийского математического общества (1962), иностранный член Американского философского общества (1961). Почётный доктор Парижского университета (1955), Стокгольмского университета (1960), Индийского статистического института  (англ.)рус. в Калькутте (1962)."""


Kolmogorov_wiki_processed = TextPreproccesing(TestTexts.Kolmogorov_wiki)
Kolmogorov_wiki_encoded = TextEncode().encode(Kolmogorov_wiki_processed.original_text)
Kolmogorov_wiki_enc_processed = TextPreproccesing(
    Kolmogorov_wiki_encoded
).count_letters_freq()
print(
    f"статья википедии с препроцессингом\n{Kolmogorov_wiki_processed.original_text[:50]}"
)
print("--" * 25)
print(
    f"зашифрованная статья википедии с препроцессингом\n{Kolmogorov_wiki_encoded[:50]}"
)


статья википедии с препроцессингом
андрей николаевич колмогоров апреля тамбов октября
--------------------------------------------------
зашифрованная статья википедии с препроцессингом
боесёк ойлпмбёгйш лпмнпдпспг брсёма убнвпг плуавса


In [8]:
def accuracy(original: str, decoded: str) -> float:
    return sum(
        [
            1
            for i in range(min(len(original), len(decoded)))
            if original[i] == decoded[i]
        ]
    ) / min(len(original), len(decoded))


In [9]:
accuracy(
    Kolmogorov_wiki_processed.original_text, Kolmogorov_wiki_enc_processed.original_text
)


0.0514839491217444

In [10]:
wnp_ru_processed.decode(Kolmogorov_wiki_enc_processed)


'ипбл г пэсфки рэз сфкуфефлфр имл кй диунфр фсдйнлй уфтсри тфр дтсэг уид уидэс фбэп эо слжмп гёэщ уид уидэсфр р си фбэп эо фтпфрфмфкфвпэсфр тфрл у ппфг д флээ р лфйдпфтд г эу мфкжз пх чжпбиу пдикяпх л ожкядидх р дфмфкфеээ е фу длээ уид уидэз тсфг кфеэс скиттэз тсфг у щипэс д флээ джлнжк пдпфтдэ д флээ ткфвпфтдэ икефлэдуфр д флээ эпчфлуицээ д флээ чжпсцэг д флээ длэефпфу длэз тсэщ лйбфр д флээ у лх д флээ млэнкэв пэй чжпсцэг д флээ упфв тдр д флээ бэчч л пцэикяпхщ жлирп пэг д флээ бэпиуэз тсэщ тэтд у чжпсцэфпикяпфу ипикэо э р лйб блжеэщ фнкитд г уид уидэсэ э  млэкфв пэг ирдфл пфридфлтсэщ линфд мф чэкфтфчээ этдфлээ у дфбфкфеээ э мл мфбирипэш уид уидэсэ эор тдпх  еф линфдх р тдидэтдэз тсфг чэоэс р зитдпфтдэ жлирп пэ бвфптфпи у ки ирлиуэ сфкуфефлфри млфч ттфл уфтсфртсфеф ефтжбилтдр ппфеф жпэр лтэд ди т бфсдфл чэоэсф уид уидэз тсэщ пижс исиб уэс исиб уээ пижс тттл мл оэб пд уфтсфртсфеф уид уидэз тсфеф фны тдри ууф р э ефбищ е лфг тфцэикэтдэз тсфеф длжби кижл ид к пэптсфг э тдикэптсфг мл уээ

Ничего не понятно. Расшифровка по частоте n-грамм с n=1 работает не очень даже на таких текстах размером с небольшую статью википедии.

# 2. Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:
 * подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
 * проведите тестирование аналогично п.1, но при помощи биграмм.
 

Самая большая и очевидная проблема, с которой я столкнулся - это то, как делать расшифровку, потому что биграммам с высокой частотностью мы доверяем больше и нужно как-то сначала их использовать, а потом на том, что останется остальные.

Или нужно как-то изначально кодировать по другому тоже сразу биграммами. Я не придумал как бороться с этой проблемой единственно верным путём, поэтому решил с ней вообще не бороться.


(+- хорошие способы, которые мне пришли в голову - это 
1. построить график скалистой осыпи для частот биграмм, отсечь по острому углу. Использовать только эти биграммы для декодирования, а остальное декодировать как в пункте 1. Но я не знаю как реализовать общее решение для всех текстов даже на 1-ом языке.
2. сначала менять только перавые биграммы слов, а потом дальше уже идти и выбирать только те биграммы, которые начинаются на конец первых и тд.
)

In [11]:
from itertools import product

alphabet_ru = "абвгдеёжзийклмнопрстуфхцчшщъыьэюя"
alphabet_en = "abcdefghijklmnopqrstuvwxyz"


class BigrammFreq:
    def __init__(self, text: str, bigramm_freq_dict: Dict[str, int]):
        self.text: str = " ".join([word for word in text.split() if len(word) > 1])
        self.bigramm_freq_dict: Dict[str, int] = bigramm_freq_dict

    def update_bigramm_freq(self, word: str) -> None:
        for i in range(len(word) - 1):
            self.bigramm_freq_dict[word[i : i + 2]] += 1

    def count_bigramm_freq(self):
        for word in self.text.split():
            self.update_bigramm_freq(word)

        self.freq_ordered_bigramms: List[str] = [
            k
            for k, v in sorted(
                self.bigramm_freq_dict.items(),
                key=lambda x: x[1],
                reverse=True,
            )
        ]
        return self

    def decode(self, another: BigrammFreq):
        decode_dict = {
            another.freq_ordered_bigramms[i]: self.freq_ordered_bigramms[i]
            for i in range(33 * 33)
        }

        def decode_word(word: str) -> str:
            return "".join(
                [decode_dict[word[i : i + 2]] for i in range(len(word) - 2)]
            )[::2]

        return " ".join(
            map(
                decode_word,
                [word for word in another.text.split()],
            )
        )


In [12]:
ru_bigramm_freq_dict_wnp_ru = {
    "".join(k): 0 for k in list(product(alphabet_ru, repeat=2))
}
ru_bigramm_freq_dict_Kolmogorov = {
    "".join(k): 0 for k in list(product(alphabet_ru, repeat=2))
}

wnp_ru_bigramm_freq = BigrammFreq(
    wnp_ru_processed.original_text,
    ru_bigramm_freq_dict_wnp_ru,
).count_bigramm_freq()
kolmogorov_bigramm_freq = BigrammFreq(
    Kolmogorov_wiki_enc_processed.original_text,
    ru_bigramm_freq_dict_Kolmogorov,
).count_bigramm_freq()


In [13]:
accuracy(
    Kolmogorov_wiki_processed.original_text,
    wnp_ru_bigramm_freq.decode(kolmogorov_bigramm_freq),
)


0.06346153846153846

Стало лучше на 1 сотую

In [14]:
wnp_ru_bigramm_freq.decode(kolmogorov_bigramm_freq)


'оо оотл ы тлканапа к  рдла исьиы ансу ж усд к кно нт  лос оы к кноот  нт  нрелаерлвтооот жл ре пр анялен  рлщ  мбов иратд шгтарн аерлвнн  ор к кно ст внн уаавс ст воо пр иаеу иленн пр ывтоенн аааприна пр ебапикше пр мбепе пр орваое ор сд ымм пр  пр ерлэи о мбепе пр ие н пр те себатдт асс о пр тевдо сд св мбепеуоватде оваи ы соен чэаа к кноо еркв о сеа елинаптсд сба  еквнжке внапр анмлвнн  рнвсиоо к кноо л нл  сбаи нрновно ст елв данленн асс  иуозжо  стсд тлканапал еа вж анстлчстн анждвмтн рен бо тс  миса елво к кно сд вп ер о ер о вп вв  в  анстлчстн к кно стн ч на б анв а жжебаивно стн оод ап  оезст нраиезст  о еенносорт  вшеуоватде ер о вп г воомозстн тпа чстн ч на асосьуяст омридст ер о вп  икозст ер о  наамвезяр ерлтоте р лт   ророзст ер о всидвн вп еенносорт   тст ер о вп рлтуст ер о вп о цаоосст тпа чст ер о вп  р ер о вп ееблтот р лт  оаптзст ер о  воомозстн к кно стн ч на еотдтстн к кно стн ч на еенносорт   ророзстн еквнжкестн ч на р лт миса омридстн бо тс  наивалтячстн бо тс 

# 3. Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:
 * предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
 * реализуйте и протестируйте его, убедитесь, что результаты улучшились.


Про MCMC читал и ориентировался на статью с Хабра

https://habr.com/ru/company/wunderfund/blog/279545/


In [23]:
from tqdm.notebook import tqdm
from nltk import everygrams


def get_ngram_freqs(text, n_gram=2):
    vocab_size = len(set(text)) ** n_gram
    if n_gram > 1:
        text = [
            "".join(ngram) for ngram in everygrams(text, min_len=n_gram, max_len=n_gram)
        ]
    freqs = {k: (v + 1) / (len(text) + vocab_size) for k, v in Counter(text).items()}
    return freqs


def get_text_proba(
    text: str,
    mapping: Dict,
    freqs: Dict,
    n_gram: int = 2,
    alphabet: str = alphabet_ru,
):
    decoded_text = "".join([mapping.get(c, " ") for c in text])
    log_proba = 0.0
    for i in range(len(decoded_text) - n_gram):
        ngram = decoded_text[i : i + n_gram]
        ngram_proba = freqs.get(ngram, 1 / (len(text) + len(alphabet) ** n_gram))
        log_proba += np.log(ngram_proba)
    return log_proba


def get_reverse_mapping_mcmc(
    enc_text,
    alphabet,
    freqs,
    n_iters=int(1e4),
    n_trials=12,
    n_gram=2,
):
    accept_num = 0
    best_reverse_mapping = None
    mapping_collection = []
    best_log_likelihood = -np.inf
    alphabet = list(alphabet)

    for trial in tqdm(range(n_trials), leave=False, position=0, total=n_trials):
        alphabet_iter = list(alphabet)
        reverse_mapping = {
            k: v for k, v in zip(alphabet, alphabet_iter[: len(alphabet)])
        }
        log_proba_current = get_text_proba(
            enc_text, reverse_mapping, freqs, n_gram=n_gram
        )

        for i in range(n_iters):
            alphabet_proposal = alphabet_iter[:]
            idx1, idx2 = np.random.choice(len(alphabet_proposal), replace=False, size=2)
            alphabet_proposal[idx1], alphabet_proposal[idx2] = (
                alphabet_proposal[idx2],
                alphabet_proposal[idx1],
            )
            reverse_mapping_proposal = {
                k: v for k, v in zip(alphabet, alphabet_proposal[: len(alphabet)])
            }
            log_proba_proposal = get_text_proba(
                enc_text, reverse_mapping_proposal, freqs, n_gram=n_gram
            )

            p_accept = np.exp(log_proba_proposal - log_proba_current)

            if p_accept > np.random.rand():
                accept_num += 1
                alphabet_iter = alphabet_proposal
                log_proba_current = log_proba_proposal
                reverse_mapping = reverse_mapping_proposal

        if log_proba_current > best_log_likelihood:
            best_log_likelihood = log_proba_current
            best_reverse_mapping = reverse_mapping

        mapping_collection.append(reverse_mapping)

    print("Likelihood: {best_log_likelihood}")
    print(f"Accept \%: {accept_num / (n_iters * n_trials)}")
    return best_reverse_mapping


In [18]:
freqs = get_ngram_freqs(
    wnp_ru_processed.original_text,
)
rev_map = get_reverse_mapping_mcmc(
    enc_text=Kolmogorov_wiki_enc_processed.original_text,
    alphabet=alphabet_ru + " ",
    freqs=freqs,
)


  0%|          | 0/12 [00:00<?, ?it/s]

Likelihood: {best_log_likelihood}
Accept \%: 0.022458333333333334


In [21]:
decoded_kolmogorov_text = "".join(
    [rev_map.get(c, " ") for c in Kolmogorov_wiki_enc_processed.original_text]
)


In [22]:
print(
    f"Accuracy:\n{accuracy(Kolmogorov_wiki_enc_processed.original_text, decoded_kolmogorov_text,)}",
    decoded_kolmogorov_text,
    sep="\n",
)


Accuracy:
0.1901877649909146
андр й нитола мия толвогором апр лу кавёом откуёру востма сом кстий вак вакит один ич трепн йжих вак вакитом м та один ич осномополознитом сомр в нной к ории м роукноск й ив полея ны бендав нкальны р челькакы м копологии г ов крии вак вакия стой логит тлассия стой в ханит к ории керёел нкноски к ории слозноски алгориквом к ории инборвашии к ории бентший к ории кригонов крия стих рудом к ории в ры к ории приёлиз ниу бентший к ории вноз скм к ории дибб р ншиальных ерамн ний к ории динавия стих сиск в бентшиональнов аналич и м руд дрегих оёласк й вак вакити и  прилоз ний амкор номакорстих раёок по билособии искории в кодологии и пр подаманиц вак вакити ичм скны  го раёокы м скакиския стой бичит м яаскноски ерамн ни дзонсона в ла амрави толвогорома проб ссор востомстого госедарскм нного еним рсик ка с доткор бичито вак вакия стих нает атад вит атад вии нает ссср пр чид нк востомстого вак вакия стого оёю скма вво м и годах г рой сошиалиския стого креда лаер ак л

# Результат очень сильно улучшился.
# Точность правильно определённых букв выросла кратно(х3) по сравнению с простым сопостовлением биграмм, отранжированных по их частотам.